In [1]:
from src.utils import EncoderDecoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.models.diffusion import SimpleDiffusionModel
from src.models.predict import KilterModel
import numpy as np
import pandas as pd

In [2]:
grades = pd.read_csv("data/raw/grades.csv", index_col=0).set_index("difficulty")["boulder_name"].to_dict()

In [3]:
model = SimpleDiffusionModel.load_from_checkpoint(
    "logs/lightning_logs/2263tvzt/checkpoints/epoch=24-step=84525.ckpt",
    config={"dim": 64, "timesteps": 1000, "lr": 1e-4, "objective" : "eps"},
).cuda()
classifier = KilterModel.load_from_checkpoint(
    "logs/lightning_logs/clq07ma5/checkpoints/epoch=51-step=15444.ckpt",
    config={"embedding_dim": 256, "dim": 512, "depth": 6, "heads": 16, "mlp_dim": 512, "dropout": 0.1},
).cuda()

In [4]:
encdec = EncoderDecoder()

In [5]:
samples = model.diffusion.sample(128)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step: 100%|██████████| 1000/1000 [02:25<00:00,  6.89it/s]


In [6]:
parsed_images = []
frames_angles = []
for i in list(samples):
    frame, angle = encdec(i)
    frames_angles.append((frame, angle))
    parsed_images.append(encdec(frame, angle))

In [7]:
image_grades = []
for image in parsed_images:
    numerical_grade = classifier.forward(image.unsqueeze(0).cuda())
    grade = grades[numerical_grade.round().long().item()]
    image_grades.append(grade)

In [14]:
n = 5
fig = make_subplots(rows=n, cols=n, horizontal_spacing=0.02, vertical_spacing=0.02, subplot_titles=[f"{image_grades[x]} @ {frames_angles[x][1]}°" for x in range(n**2)])
for i in range(n):
    for j in range(n):
        climb, angle = frames_angles[i*n + j]
        fig.add_trace(go.Image(z=encdec.plot_climb(climb)), row=i+1, col=j+1)

for i in range(n):
    for j in range(n):
        fig.update_xaxes(showticklabels=False, row=i + 1, col=j + 1)
        fig.update_yaxes(showticklabels=False, row=i + 1, col=j + 1)
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(t=25,b=5, l=5, r=5),
)
fig.write_image(f"figs/{n}x{n}_samples.png", scale=4)
# fig.show()

In [21]:
frames = []
angles = []
for tup in frames_angles:
    frames.append(tup[0])
    angles.append(tup[1])

In [23]:
df = pd.DataFrame()
df["frames"] = frames
df["angle"] = angles
df["grade"] = image_grades

In [26]:
df.to_csv("test.csv", index=False)